### Day 2: 
#### Section 1: tools

#### ADK Best Practices in Action

1. Dictionary Returns: Tools return {"status": "success", "data": ...} or {"status": "error", "error_message": ...}
2. Clear Docstrings: LLMs use docstrings to understand when and how to use tools
3. Type Hints: Enable ADK to generate proper schemas (str, dict, etc.)
4. Error Handling: Structured error responses help LLMs handle failures gracefully

In [ ]:
import os

gemini_key = "xxx"

os.environ["GOOGLE_API_KEY"] = gemini_key

In [3]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

### How to define a Tool?
**Any Python function can become an agent tool by following these simple guidelines**:

Create a Python function 

Follow the best practices listed below 

Add your function to the agent's tools=[] list and ADK handles the rest automatically. 

### ADK Best Practices in Action
Notice how our tools follow ADK best practices:


1. Dictionary Returns: Tools return {"status": "success", "data": ...} or {"status": "error", "error_message": ...}
2. Clear Docstrings: LLMs use docstrings to understand when and how to use tools
3. Type Hints: Enable ADK to generate proper schemas (str, dict, etc.)
4. Error Handling: Structured error responses help LLMs handle failures gracefully

In [5]:
### example of the first tool
# Example: Currency Converter Agent¶
# This agent can convert currency from one denomination to another and calculates the fees to do the conversion. The agent has two custom tools and follows the workflow:

# Fee Lookup Tool - Finds transaction fees for the conversion (mock)
# Exchange Rate Tool - Gets currency conversion rates (mock)
# Calculation Step - Calculates the total conversion cost including the fees


# Pay attention to the docstring, type hints, and return value.
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


In [6]:
# Second tool get_exchange_rate
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


In [7]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [8]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > You will receive 455.7 Euros.

Here's how that's calculated:
1.  **Fee:** Your Platinum Credit Card has a 2.0% fee. This means a fee of 10.00 USD (2.0% of 500 USD).
2.  **Amount after fee:** After the fee, you have 490.00 USD remaining (500 USD - 10.00 USD).
3.  **Exchange:** The current exchange rate is 0.93 EUR per USD.
4.  **Final amount:** Converting 490.00 USD at this rate gives you 455.70 EUR (490.00 USD * 0.93).


### Improving Agent Reliability with Code
#### Use agents to generate python code/functions

In [ ]:
# a calculator agent
# use ADK Built-in Code Executor
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

In [15]:
# Update the Agent's instruction and toolset
# """
# 1. Update the currency_agent's instructions to generate Python code
# Original: "Calculate the final amount after fees" (vague math instructions)
# Enhanced: "Generate a Python code to calculate the final amount .. and use the calculation_agent to run the code and compute final amount"

# ADK lets you use any agent as a tool using AgentTool.

# 2.Add AgentTool(agent=calculation_agent) to the tools list
# The specialist agent appears as a callable tool to the root agent
# """

enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [16]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [17]:
# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.
enhanced_currency_agent > The final amount is 103199.10 INR.

Here's the breakdown of the calculation:
*   The transaction fee for a Bank Transfer is 1% of the amount being sent.
*   The fee amount is 12.50 USD (1% of 1,250 USD).
*   After deducting the fee, the amount remaining is 1,237.50 USD.
*   The exchange rate applied is 83.58 INR per USD.
*   Therefore, the final converted amount is 103,199.10 INR (1,237.50 USD \* 83.58 INR/USD).


In [13]:
## helper function
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [18]:
show_python_code_and_result(response)

Generated Python Code >>  
usd_amount = 1250
fee_percentage = 0.01
exchange_rate = 83.58

fee_amount = usd_amount * fee_percentage
amount_after_fee = usd_amount - fee_amount
final_inr_amount = amount_after_fee * exchange_rate

print(f"USD Amount: {usd_amount}")
print(f"Fee Percentage: {fee_percentage*100}%")
print(f"Fee Amount: {fee_amount:.2f} USD")
print(f"Amount after Fee: {amount_after_fee:.2f} USD")
print(f"Exchange Rate: {exchange_rate} INR/USD")
print(f"Final Converted Amount: {final_inr_amount:.2f} INR")




### Agents tool best practices
#### Section 2: MCP


In [19]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [20]:
# Create the MCP toolset
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [21]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

In [22]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [26]:
# test the agent
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Continue session: debug_session_id

User > Provide a sample tiny image
image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > Here is a tiny image.


In [27]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

In [28]:
# Try kaggle MCP server
mcp_kaggle_server =McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "mcp-remote", "https://www.kaggle.com/mcp"],
        ),
        timeout=30,
    )
)

In [30]:
kaggle_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="kaggle_agent",
    instruction="Use the MCP Tool to interact with Kaggle datasets, notebooks, and competitions.",
    tools=[mcp_kaggle_server],
)

In [32]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=kaggle_agent)
response = await runner.run_debug(
    "What are the top 5 most Votes datasets on Kaggle?", verbose=True
)


 ### Created new session: debug_session_id

User > What are the top 5 most Votes datasets on Kaggle?
kaggle_agent > [Calling tool: search_datasets({'request': {'pageSize': 5, 'sortBy': 'Votes'}})]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "datasets": [\n    {\n      "id": 661950,\n      "ref": ...]
kaggle_agent > The top 5 most voted datasets on Kaggle are:

1.  **Credit Card Fraud Detection** with 12,770 votes.
2.  **COVID-19 Open Research Dataset Challenge (CORD-19)** with 11,270 votes.
3.  **Netflix Movies and TV Shows** with 9,571 votes.
4.  **Chest X-Ray Images (Pneumonia)** with 7,311 votes.
5.  **Video Game Sales** with 6,651 votes.
